In [16]:
import pandas as pd
import csv
import re

In [2]:
records_dataframe = pd.read_pickle('working_files/records_dataframe.pkl')

In [3]:
records_dataframe.head()

,Model,Year Range,Complete Records String,Complete Records,VIN,User,Case Number,EDR Data Imaging Date,Filename,Saved On,Imaged with CDR Version:,Reported with CDR Version:,Pre-Crash Data
2,E-450,2004-2022,...,Complet...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None,"Friday, April 29 2022 at 13:07:54",Crash Data Retrieval Tool 21.4,Crash Data Retrieval Tool 21.5,"\nTimes (sec),""Speed, vehicle indicated MPH"",""..."
4,E-450,2004-2022,...,Complet...,1FDXE4FSXCDA06364,SP,866715-2017,08/07/2017,None,"Monday, August 7 2017 at 12:13:58",Crash Data Retrieval Tool 17.4,Crash Data Retrieval Tool 23.0,"\nTimes (sec),""Speed, vehicle indicated MPH"",""..."
6,Transit-150,2015-2022,...,Complet...,1FTNR1ZM1FKB19874,SP,685630-2015,11/03/2015,1FTNR1ZM1FKB19874_ACM.CDRX,None,Crash Data Retrieval Tool 16.1,Crash Data Retrieval Tool 21.5,"\nTimes (sec),""Speed, Vehicle Indicated (km/h)..."
17,1500,2013-2022,...,Complet...,3C6JR6DT3KG546976,M. Mikhailov,01869039-2020,07/24/2020,3C6JR6DT3KG546976_ACM.CDRX,"Friday, July 24 2020 at 14:26:34",Crash Data Retrieval Tool 19.4,Crash Data Retrieval Tool 21.5,NaT
19,Silverado 1500,2019-2022,...,Complet...,3GCNWAEF5LG293370,M. Mikhailov,02083717-2020,01/27/2021,3GCNWAEF5LG293370_ACM.CDRX,None,Crash Data Retrieval Tool 21.0,Crash Data Retrieval Tool 21.5,None


This code takes the Pre-Crash data string and converts it to table format.

In [4]:
data = records_dataframe['Pre-Crash Data'].iloc[1]

# create a list of rows from the data
rows = data.split('\n')

# get the column names from the first row
# variable_names = next(csv.reader([rows[1]]))

event_1_pre_crash_data = []
for i in range(1, len(rows)):
        event_1_pre_crash_data.append(next(csv.reader([rows[i]])))

analysis_table_dataframe = pd.DataFrame(event_1_pre_crash_data, columns = event_1_pre_crash_data[0])
analysis_table_dataframe

,Times (sec),"Speed, vehicle indicated MPH","Accelerator pedal, % full","Service brake, on/off",Engine RPM,"ABS activity (engaged, non-engaged)","Stability control (engaged, non-engaged)","Traction Control via Brakes (engaged, non-engaged)","Traction Control via Engine (engaged, non-engaged)"
0,Times (sec),"Speed, vehicle indicated MPH","Accelerator pedal, % full","Service brake, on/off",Engine RPM,"ABS activity (engaged, non-engaged)","Stability control (engaged, non-engaged)","Traction Control via Brakes (engaged, non-enga...","Traction Control via Engine (engaged, non-enga..."
1,...,72.7,83,Off,"4,100",non-engaged,non-engaged,non-engaged,non-engaged
2,...,79.5,94,Off,"4,500",non-engaged,non-engaged,non-engaged,non-engaged
3,...,73.9,100,Off,"3,700",non-engaged,non-engaged,non-engaged,non-engaged
4,...,70.2,100,Off,"3,900",non-engaged,non-engaged,non-engaged,non-engaged
5,...,73.3,100,Off,"4,100",non-engaged,non-engaged,non-engaged,non-engaged
6,...,72.7,100,Off,"4,000",non-engaged,non-engaged,non-engaged,non-engaged
7,...,73.3,100,Off,"4,000",non-engaged,non-engaged,non-engaged,non-engaged
8,...,70.8,100,Off,"4,000",non-engaged,non-engaged,non-engaged,non-engaged
9,...,65.2,100,Off,"3,600",engaged,non-engaged,non-engaged,non-engaged


In [23]:
# reset the DataFrame index to avoid errors during iteration and remove empty rows that don't contain records
records_dataframe.reset_index(drop=True, inplace=True)

# initialize an empty dataframe for each unique VIN
unique_vins = records_dataframe['VIN'].unique()
pre_crash_dataframes = {vin: pd.DataFrame() for vin in unique_vins}

for i in range(len(records_dataframe)):
    # get the Pre-Crash Data for the current row
    data = records_dataframe['Pre-Crash Data'].iloc[i]

    # check if the data is a string to prevent string errors
    if not isinstance(data, str):
        # if not, set the value to NaN and skip to the next iteration
        continue

    # create a list of rows from the data
    rows = data.split('\n')

    event_1_pre_crash_data = []
    for j in range(1, len(rows)):
        try:
            # use a try/except block to catch any errors
            event_1_pre_crash_data.append(next(csv.reader([rows[j]])))
        except csv.Error:
            # if an error occurs, skip to the next iteration
            continue

    # save the DataFrame to the pre_crash_dataframes dictionary with the corresponding VIN as the key
    vin = records_dataframe['VIN'].iloc[i]
    pre_crash_dataframes[vin] = pd.DataFrame(event_1_pre_crash_data, columns=event_1_pre_crash_data[0])

# add the DataFrame dictionary to the records_dataframe
records_dataframe['Pre-Crash Dataframes'] = [pre_crash_dataframes[vin] for vin in records_dataframe['VIN']]


In [24]:
records_dataframe

,Model,Year Range,Complete Records String,Complete Records,VIN,User,Case Number,EDR Data Imaging Date,Filename,Saved On,Imaged with CDR Version:,Reported with CDR Version:,Pre-Crash Data,Pre-Crash Dataframes
0,E-450,2004-2022,...,Complet...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None,"Friday, April 29 2022 at 13:07:54",Crash Data Retrieval Tool 21.4,Crash Data Retrieval Tool 21.5,"\nTimes (sec),""Speed, vehicle indicated MPH"",""...",Time...
1,E-450,2004-2022,...,Complet...,1FDXE4FSXCDA06364,SP,866715-2017,08/07/2017,None,"Monday, August 7 2017 at 12:13:58",Crash Data Retrieval Tool 17.4,Crash Data Retrieval Tool 23.0,"\nTimes (sec),""Speed, vehicle indicated MPH"",""...",Time...
2,Transit-150,2015-2022,...,Complet...,1FTNR1ZM1FKB19874,SP,685630-2015,11/03/2015,1FTNR1ZM1FKB19874_ACM.CDRX,None,Crash Data Retrieval Tool 16.1,Crash Data Retrieval Tool 21.5,"\nTimes (sec),""Speed, Vehicle Indicated (km/h)...",Time...
3,1500,2013-2022,...,Complet...,3C6JR6DT3KG546976,M. Mikhailov,01869039-2020,07/24/2020,3C6JR6DT3KG546976_ACM.CDRX,"Friday, July 24 2020 at 14:26:34",Crash Data Retrieval Tool 19.4,Crash Data Retrieval Tool 21.5,NaT,Empty DataFrame Columns: [] Index: []
4,Silverado 1500,2019-2022,...,Complet...,3GCNWAEF5LG293370,M. Mikhailov,02083717-2020,01/27/2021,3GCNWAEF5LG293370_ACM.CDRX,None,Crash Data Retrieval Tool 21.0,Crash Data Retrieval Tool 21.5,None,Empty DataFrame Columns: [] Index: []
5,Silverado 1500,2019-2022,...,Complet...,3GCNWAEF5LG401485,M. Mikhailov,02146598-2021,03/29/2021,3GCNWAEF5LG401485_ACM.CDRX,"Monday, March 29 2021 at 13:56:50",Crash Data Retrieval Tool 21.0,Crash Data Retrieval Tool 21.5,None,Empty DataFrame Columns: [] Index: []
6,None,None,...,Complete Record 0 Claim ...,None,None,None,10/31/2018,None,None,None,None,NaT,Empty DataFrame Columns: [] Index: []
